In [1]:
setwd("~/0-workspace/CCR7_DC/oral-tolerance-Colonna/")

suppressPackageStartupMessages({
library(Seurat) # v4.4
library(Matrix)
library(future)
library(dplyr)
library(ggplot2)
library(cowplot)
library(ComplexHeatmap)
library(circlize)
library(ggrepel)
})
set.seed(1)
options(future.globals.maxSize = Inf)

pal <- readRDS("plots/palette.rds")
source("../utils.R")

In [2]:
cols <- rev(RColorBrewer::brewer.pal(11,"Spectral"))
transitions <- c(0, 15, 25, 30, 40, 50, 60, 70, 75, 85, 100)
scaled_transitions <- scales::rescale(transitions, from = c(0, 100), to = c(0, 1))
scale.color <- scale_color_gradientn(colors = cols, values = scaled_transitions)

In [3]:
RenameGenesSeurat <- function(obj, newnames) { # Replace gene names in different slots of a Seurat object. Run this before integration. It only changes obj@assays$RNA@counts, @data and @scale.data.
  print("Run this before integration. It only changes obj@assays$RNA@counts, @data and @scale.data.")
  RNA <- obj@assays$RNA
  
  if (nrow(RNA) == length(newnames)) {
    if (length(RNA@counts)) RNA@counts@Dimnames[[1]] <- newnames
    if (length(RNA@data)) RNA@data@Dimnames[[1]] <- newnames
    if (length(RNA@scale.data)) rownames(RNA@scale.data) <- newnames
    if (length(RNA@meta.features)) {
      RNA@meta.features$prev.names <- rownames(RNA@meta.features)
      rownames(RNA@meta.features) <- newnames
    }
    if (length(RNA@var.features)){
      RNA@var.features <- rownames(RNA@meta.features[match(RNA@var.features, RNA@meta.features$prev.names), ])
    }
  } else {"Unequal gene sets: nrow(RNA) != nrow(newnames)"}
  obj@assays$RNA <- RNA
  return(obj)
}

In [4]:
select.markers <- function(fn, output.dir = 'results/markers/', pairwise = F, fc.thr = 1.5, apv.thr = 0.01, n = Inf, exCC = F, cc.genes = NULL){
  markers <- read.csv(file = paste0(output.dir, fn, ".csv"), header = T, stringsAsFactors = F) %>%
    subset(!grepl("(^MT-)|(^RPS)|(^RPL)|(^MRPL)|(^MRPS)", toupper(gene)))
  if (exCC){
    markers <- markers[!(toupper(markers$gene) %in% toupper(cc.genes)),]
  }
  if(pairwise){
    markers <- subset(markers, abs(avg_log2FC) > log2(fc.thr) & p_val_adj < apv.thr)
    markers$cluster <- ifelse(markers$avg_log2FC > 0, "up", "down")
    if(!is.infinite(n)) markers <- markers %>% group_by(cluster) %>% top_n(n, abs(avg_log2FC))
    markers <- markers[order(markers$avg_log2FC), ]
  } else{
    markers <- subset(markers, avg_log2FC > log2(fc.thr) & p_val_adj < apv.thr)
    if(any(duplicated(markers$gene))) markers <- markers %>% group_by(gene) %>% top_n(1, avg_log2FC)
    if(!is.infinite(n)) markers <- markers %>% group_by(cluster) %>% top_n(n, avg_log2FC)
    markers <- markers[order(markers$cluster, -markers$avg_log2FC), ]
  }
  return(markers)
}

# Load SRO

In [5]:
# load SRO ####
sro.ln <- readRDS("../TC_all_LN/results/SRO_subset.rds")

In [36]:
pref.sro <- "Seurat/merged-4I/"; pref.p.sro <- "plots/Seurat/merged-4I/"
sro <- readRDS(paste0(pref.sro, "SRO.rds"))
sro <- RenameGenesSeurat(sro, toupper(rownames(sro)))

[1] "Run this before integration. It only changes obj@assays$RNA@counts, @data and @scale.data."


## TC module scores

In [10]:
pref.p.sro
dir.create(paste0(pref.p.sro, "module-scores"))

[1] "plots/Seurat/merged-4I/"

Warning message in dir.create(paste0(pref.p.sro, "module-scores")):
“'plots/Seurat/merged-4I/module-scores' already exists”


In [11]:
# load markers ####
markers.ln <- select.markers("TC-clusters", output.dir = paste0('../TC_all_LN/results/markers/'))

In [12]:
tc1.markers.ln <- markers.ln[markers.ln$cluster == 3,]
tc2.markers.ln <- markers.ln[markers.ln$cluster == 2,]
tc3.markers.ln <- markers.ln[markers.ln$cluster == 1,]
tc4.markers.ln <- markers.ln[markers.ln$cluster == 5,]
ki_tc.markers.ln <- markers.ln[markers.ln$cluster == 7,]
transitional.markers.ln <- markers.ln[markers.ln$cluster == 4,]

In [13]:
sum(toupper(tc1.markers.ln$gene) %in% rownames(sro))
sum(toupper(tc2.markers.ln$gene) %in% rownames(sro))
sum(toupper(tc3.markers.ln$gene) %in% rownames(sro))
sum(toupper(tc4.markers.ln$gene) %in% rownames(sro))
sum(toupper(ki_tc.markers.ln$gene) %in% rownames(sro))
sum(toupper(transitional.markers.ln$gene) %in% rownames(sro))

[1] 152

[1] 153

[1] 199

[1] 217

[1] 289

[1] 83

In [14]:
sro <- AddModuleScore(sro, list(toupper(transitional.markers.ln$gene)), name = "transitional.markers")
sro <- AddModuleScore(sro, list(toupper(ki_tc.markers.ln$gene)), name = "ki_tc.markers")
sro <- AddModuleScore(sro, list(toupper(tc1.markers.ln$gene)), name = "tc1.markers")
sro <- AddModuleScore(sro, list(toupper(tc2.markers.ln$gene)), name = "tc2.markers")
sro <- AddModuleScore(sro, list(toupper(tc3.markers.ln$gene)), name = "tc3.markers")
sro <- AddModuleScore(sro, list(toupper(tc4.markers.ln$gene)), name = "tc4.markers")

In [15]:
f.w <- 12; f.h <- 10

In [32]:
pdf(paste0(pref.p.sro, "module-scores/UMAP-transitional_markers-from-TC_all_LN.pdf"), width = f.w, height = f.h)
print(plot.continuous.value(
    sro, scale.color = scale.color,
    vis = sro@reductions$umap@cell.embeddings,
    idx = rownames(sro@meta.data), point.size = 1,
    val = get.named.vector.sro(sro, paste0("transitional.markers1")), val.name = 'module\nscore') + ggtitle("transitional.markers"))
dev.off()

pdf(paste0(pref.p.sro, "module-scores/UMAP-TCI_markers-from-TC_all_LN.pdf"), width = f.w, height = f.h)
print(plot.continuous.value(
    sro, scale.color = scale.color,
    vis = sro@reductions$umap@cell.embeddings,
    idx = rownames(sro@meta.data), point.size = 1,
    val = get.named.vector.sro(sro, paste0("tc1.markers1")), val.name = 'module\nscore') + ggtitle("TC I markers"))
dev.off()

pdf(paste0(pref.p.sro, "module-scores/UMAP-TCII_markers-from-TC_all_LN.pdf"), width = f.w, height = f.h)
print(plot.continuous.value(
    sro, scale.color = scale.color,
    vis = sro@reductions$umap@cell.embeddings,
    idx = rownames(sro@meta.data), point.size = 1,
    val = get.named.vector.sro(sro, paste0("tc2.markers1")), val.name = 'module\nscore') + ggtitle("TC II markers"))
dev.off()

pdf(paste0(pref.p.sro, "module-scores/UMAP-TCIII_markers-from-TC_all_LN.pdf"), width = f.w, height = f.h)
print(plot.continuous.value(
    sro, scale.color = scale.color,
    vis = sro@reductions$umap@cell.embeddings,
    idx = rownames(sro@meta.data), point.size = 1,
    val = get.named.vector.sro(sro, paste0("tc3.markers1")), val.name = 'module\nscore') + ggtitle("TC III markers"))
dev.off()

pdf(paste0(pref.p.sro, "module-scores/UMAP-TCIV_markers-from-TC_all_LN.pdf"), width = f.w, height = f.h)
print(plot.continuous.value(
    sro, scale.color = scale.color,
    vis = sro@reductions$umap@cell.embeddings,
    idx = rownames(sro@meta.data), point.size = 1,
    val = get.named.vector.sro(sro, paste0("tc4.markers1")), val.name = 'module\nscore') + ggtitle("TC IV markers"))
dev.off()

pdf(paste0(pref.p.sro, "module-scores/UMAP-KI_TC_markers-from-TC_all_LN.pdf"), width = f.w, height = f.h)
print(plot.continuous.value(
    sro, scale.color = scale.color,
    vis = sro@reductions$umap@cell.embeddings,
    idx = rownames(sro@meta.data), point.size = 1,
    val = get.named.vector.sro(sro, paste0("ki_tc.markers1")), val.name = 'module\nscore') + ggtitle("Ki67+TC markers"))
dev.off()

pdf 
  2

pdf 
  2

pdf 
  2

pdf 
  2

pdf 
  2

pdf 
  2

In [35]:
module.scores <- sro@meta.data[grep("markers1", colnames(sro@meta.data), value = T)]

In [39]:
dir.create(paste0(pref.sro, "module-scores/"))

In [40]:
write.csv(module.scores, paste0(pref.sro, "module-scores/TC_all_LN-scores.csv"), quote = F)

In [16]:
module.scores <- sro@meta.data[grep("markers1", colnames(sro@meta.data), value = T)]

In [17]:
pl <- lapply(colnames(module.scores), function(module){
    module.name <- stringr::str_split_i(module, '.marker', i = 1)
    p <- plot.continuous.value(
        sro, scale.color = scale.color,
        vis = sro@reductions$umap@cell.embeddings,
        idx = rownames(sro@meta.data), point.size = 1,
        val = get.named.vector.sro(sro, paste0(module)), val.name = 'module\nscore') + ggtitle(module.name)
    return(p)
})

In [18]:
numcol <- 3
numrow <- ceiling(length(pl)/numcol)
pdf(paste0(pref.p.sro, "module-scores/UMAP-TC.markers.pdf"), width = 8*numcol, height = 6*numrow)
print(plot_grid(plotlist = pl, align = 'hv', ncol = 3))
dev.off()

pdf 
  2

## TC UCell scores

In [25]:
library(UCell)

In [29]:
markers.ln <- markers.ln[markers.ln$gene %in% rownames(sro),]

In [31]:
tc1.markers.ln <- markers.ln[markers.ln$cluster == 3,]
tc2.markers.ln <- markers.ln[markers.ln$cluster == 2,]
tc3.markers.ln <- markers.ln[markers.ln$cluster == 1,]
tc4.markers.ln <- markers.ln[markers.ln$cluster == 5,]
ki_tc.markers.ln <- markers.ln[markers.ln$cluster == 7,]
transitional.markers.ln <- markers.ln[markers.ln$cluster == 4,]

In [40]:
gene_sets <- list(
    transitional = toupper(transitional.markers.ln$gene),
    ki_tc = toupper(ki_tc.markers.ln$gene),
    tc1 = toupper(tc1.markers.ln$gene),
    tc2 = toupper(tc2.markers.ln$gene),
    tc3 = toupper(tc3.markers.ln$gene),
    tc4 = toupper(tc4.markers.ln$gene)
)

In [41]:
sro <- AddModuleScore_UCell(sro, gene_sets, ncores = 8)

In [42]:
module.scores <- sro@meta.data[grep("UCell", colnames(sro@meta.data), value = T)]

In [43]:
head(module.scores)

,transitional_UCell,ki_tc_UCell,tc1_UCell,tc2_UCell,tc3_UCell,tc4_UCell
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
hCD2_neg_AAACCATTCCATGCAA-1,0.07873882,0.07646037,0.1650144,0.0930098,0.3858650,0.5337158
hCD2_neg_AAACCCTGTTGCACAT-1,0.22510205,0.10077759,0.1425485,0.1233172,0.1203823,0.1787975
hCD2_neg_AAACCGCTCAGCTCGG-1,0.23579916,0.11053384,0.2628367,0.2233544,0.1753643,0.2031907
hCD2_neg_AAACGAATCAAGTTCG-1,0.09945543,0.05335232,0.1614442,0.1111088,0.2948241,0.4259244
hCD2_neg_AAACGAATCTGTGAAT-1,0.24312063,0.09772852,0.1275465,0.1183544,0.1175287,0.1509556
hCD2_neg_AAACGATGTAAGTCAG-1,0.21553705,0.12279396,0.1112297,0.1196542,0.1162796,0.1507966


In [44]:
pl <- lapply(colnames(module.scores), function(module){
    module.name <- stringr::str_split_i(module, '_UCell', i = 1)
    p <- plot.continuous.value(
        sro, scale.color = scale.color,
        vis = sro@reductions$umap@cell.embeddings,
        idx = rownames(sro@meta.data), point.size = 1,
        val = get.named.vector.sro(sro, paste0(module)), val.name = 'UCell\nscore') + ggtitle(module.name)
    return(p)
})

In [45]:
numcol <- 3
numrow <- ceiling(length(pl)/numcol)
pdf(paste0(pref.p.sro, "module-scores/UMAP-UCell.score-TC.markers.pdf"), width = 8*numcol, height = 6*numrow)
print(plot_grid(plotlist = pl, align = 'hv', ncol = 3))
dev.off()

pdf 
  2

## TC average expression

In [19]:
AddModuleExpression <- function(
        object, 
        features, 
        name = 'module',
        assay = 'RNA', 
        slot = 'data'){
    
    assay.old <- DefaultAssay(object = object)
    assay <- assay %||% assay.old
    DefaultAssay(object = object) <- assay
    assay.data <- GetAssayData(object = object, assay = assay, slot = slot)
    features.old <- features
    
    if (is.null(x = features)) {
        stop("Missing input feature list")
    }
    
    x <- features
    missing.features <- setdiff(x = x, y = rownames(x = object))
    if (length(x = missing.features) > 0) {
        warning(
            "The following features are not present in the object: ",
            paste(missing.features, collapse = ", "),
            call. = FALSE,
            immediate. = TRUE
        )
    }
    features <- intersect(x = x, y = rownames(x = object))
    module.length <- length(x = features)
    if (module.length == 0){
        stop(
            "Not enough features in the data",
            "exiting..."
        )
    }
    
    features.use <- features
    data.use <- assay.data[features.use, , drop = FALSE]
    data.use <- expm1(x = data.use)
    
    module.expression <- Matrix::colMeans(x = data.use)
    module.expression <- log1p(module.expression)
    
    object@meta.data[[name]] <- module.expression[rownames(object@meta.data)]
    DefaultAssay(object = object) <- assay.old
    return(object)
}

In [20]:
tc1.markers.ln <- markers.ln[markers.ln$cluster == 3,]
tc2.markers.ln <- markers.ln[markers.ln$cluster == 2,]
tc3.markers.ln <- markers.ln[markers.ln$cluster == 1,]
tc4.markers.ln <- markers.ln[markers.ln$cluster == 5,]
ki_tc.markers.ln <- markers.ln[markers.ln$cluster == 7,]
transitional.markers.ln <- markers.ln[markers.ln$cluster == 4,]

In [21]:
sro <- AddModuleExpression(sro, toupper(transitional.markers.ln$gene), name = "transitional.avg.expr")
sro <- AddModuleExpression(sro, toupper(ki_tc.markers.ln$gene), name = "ki_tc.avg.expr")
sro <- AddModuleExpression(sro, toupper(tc1.markers.ln$gene), name = "tc1.avg.expr")
sro <- AddModuleExpression(sro, toupper(tc2.markers.ln$gene), name = "tc2.avg.expr")
sro <- AddModuleExpression(sro, toupper(tc3.markers.ln$gene), name = "tc3.avg.expr")
sro <- AddModuleExpression(sro, toupper(tc4.markers.ln$gene), name = "tc4.avg.expr")

Warning message:
“The `slot` argument of `GetAssayData()` is deprecated as of SeuratObject 5.0.0.
ℹ Please use the `layer` argument instead.”


In [22]:
module.scores <- sro@meta.data[grep("avg.expr", colnames(sro@meta.data), value = T)]

In [23]:
pl <- lapply(colnames(module.scores), function(module){
    module.name <- stringr::str_split_i(module, '.avg.expr', i = 1)
    p <- plot.continuous.value(
        sro, scale.color = scale.color,
        vis = sro@reductions$umap@cell.embeddings,
        idx = rownames(sro@meta.data), point.size = 1,
        val = get.named.vector.sro(sro, paste0(module)), val.name = 'average\nexpression') + ggtitle(module.name)
    return(p)
})

In [24]:
numcol <- 3
numrow <- ceiling(length(pl)/numcol)
pdf(paste0(pref.p.sro, "module-scores/UMAP-avg.expr-TC.markers.pdf"), width = 8*numcol, height = 6*numrow)
print(plot_grid(plotlist = pl, align = 'hv', ncol = 3))
dev.off()

pdf 
  2